In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import basedosdados as bd
import re
import json
import string
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import scipy.stats as stats
import os

In [ ]:
municipios = pd.read_csv("csv/municipios.csv")
zonas_eleitorais = pd.read_csv("csv/zona_eleitoral.csv", error_bad_lines=False, delimiter=',')

zonas_eleitorais = zonas_eleitorais[zonas_eleitorais['uf'].eq('RJ') & zonas_eleitorais['nome_municipio'].eq('RIO DE JANEIRO')]
zonas_eleitorais['id'].replace('(19-0*)','',regex=True, inplace = True)

orientacao_partidos = pd.read_csv("csv/orientacao_partidos.csv", error_bad_lines=False, delimiter=',')
orientacao_partidos =  orientacao_partidos.replace({ "—":"esquerda"})
orientacao_partidos['Nome'] = orientacao_partidos['Nome'].map(lambda x: x.upper())

with open('csv/bairros-geojson.json', encoding="utf8") as geojson:
    bairros = json.load(geojson)

In [ ]:
def dataAlreadyBeenGot(csvName):
    if os.stat(csvName).st_size == 0:
        return False
    else:
        return True

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    queryResultadosPartidoMunicipio = """
        SELECT MZ.ano, MZ.zona, MZ.sigla_partido, MZ.cargo, MZ.turno, MZ.votos_nominais, MZ.votos_nao_nominais FROM `basedosdados.br_tse_eleicoes.resultados_partido_municipio_zona` AS MZ
        WHERE MZ.id_municipio_tse={} AND MZ.sigla_uf='RJ'
        ORDER BY MZ.ano ASC""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0])

    dfResultadosPartidoMunicipio = bd.read_sql(queryResultadosPartidoMunicipio, billing_project_id='capacitacaoufrjanalytica')
    dfResultadosPartidoMunicipio.to_csv("csv/resultado_eleicoes.csv", index=False)
else:
    dfResultadosPartidoMunicipio = pd.read_csv("csv/resultado_eleicoes.csv")

In [ ]:
dfResultadosPartidoMunicipio['sigla_partido'].replace({'PEN':'PATRIOTA', 'PATRI':'PATRIOTA', 
                                                       'PT DO B':'AVANTE', 'PT do B':'AVANTE', 
                                                       'PC do B': 'PCDOB', 'PC DO B': 'PCDOB',
                                                       'PRONA':'PL', 'PGT':'PL', 'PPB':'PP',
                                                       'PPS': 'CIDADANIA', 'PRN':'PTC',
                                                       'PRP': 'PATRIOTA', 'PSDC':'DC',
                                                       'PSN':'PODE', 'PST': 'PL', 'PTN':'PODE',
                                                       'PHS':'PODE', 'PRB':'REPUBLICANOS',
                                                       'PR': 'PL', 'PPL': 'PCDOB', 'SD': 'SOLIDARIEDADE',
                                                       'PAN':'PTB','PFL':'DEM', 'PMDB':'MDB'}, inplace=True)


In [ ]:
dfResultadosPartidoMunicipio = dfResultadosPartidoMunicipio.set_index(['ano', 'sigla_partido', 'zona', 'cargo', 'turno']).groupby(level=(0, 1, 2, 3, 4)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

In [ ]:
votosTotaisPorPartido = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPorPartido['votos totais'] = votosTotaisPorPartido['votos_nominais'] + votosTotaisPorPartido['votos_nao_nominais']

In [ ]:
partido = ['PT', 'PSL', 'PSOL']
fig = px.line(votosTotaisPorPartido[votosTotaisPorPartido['sigla_partido'].isin(partido)], x="ano", y="votos totais", 
              color='sigla_partido',
              title='Votos dos partido '+str(partido)[1:-1]+' X ano')
fig.show()

In [ ]:
def returnOrientacaoPolitica(partido):
    return orientacao_partidos[orientacao_partidos['Nome'].eq(partido)]['Congresso em Foco (2019)[37]'].values[0]

In [ ]:
dfResultadosPartidoMunicipio['orientacao politica'] = dfResultadosPartidoMunicipio['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

In [ ]:
votosPorOrientacaoPolitica = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosPorOrientacaoPolitica['votos totais'] = votosPorOrientacaoPolitica['votos_nominais'] + votosPorOrientacaoPolitica['votos_nao_nominais']

In [ ]:
fig = px.line(votosPorOrientacaoPolitica, x="ano", y="votos totais", color='orientacao politica',
             title='Votos de determinada ideologica polìtica X ano')
fig.show()

In [ ]:
def neighhboorhoodCoordinateIsInside(latitude, longitude):
    for e in bairros['features']:
        if (e['geometry']['type'] != 'MultiPolygon'):
            vertices = e['geometry']['coordinates'][0]
        else:
            vertices = e['geometry']['coordinates'][0][0]
        wn = 0;
        i = 0
        j = len(vertices) - 1
        while (i < len(vertices)):
            v = vertices[j];
            v1 = vertices[i];
            if ( v[0] <= longitude ) :
                if ( longitude < v1[0] ):  
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) > 0 ):
                        wn += 1;
            else:
                if( longitude >= v1[0] ):
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) < 0 ):
                        wn -= 1;
            j = i
            i += 1
    
        if (wn != 0):
            return e['properties']['NOME']
        
def returnLat(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['latitude'].values[0] 
def returnLon(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['longitude'].values[0] 
def returnBairro(zona):
    return zonas_eleitorais[zonas_eleitorais['id'].eq(str(zona))]['bairro'].values[0]

In [ ]:
dfResultadosPartidoMunicipio['bairro'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnBairro(x))

In [ ]:
dfResultadosPartidoMunicipio['latitude'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnLat(x))
dfResultadosPartidoMunicipio['longitude'] = dfResultadosPartidoMunicipio['zona'].map(lambda x: returnLon(x))

In [ ]:
auxDf = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)]

def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_votosArray = []
    partidosArray = []
    latitudeArray = []
    longitudeArray = []
    for bairro, frame in auxDf[auxDf['ano'].eq(ano)].groupby('bairro'):
        n_votos = np.max(frame['votos_nominais'])
        partido = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['sigla_partido'].values[0]
        latitude = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['latitude'].values[0]
        longitude = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro)]['longitude'].values[0]
        
        n_votosArray.append(n_votos)
        partidosArray.append(partido)
        latitudeArray.append(latitude)
        longitudeArray.append(longitude)
        bairrosArray.append(neighhboorhoodCoordinateIsInside(latitude, longitude))

    dfAgrupadoResultadosPartidoMunicipio = pd.DataFrame([bairrosArray, n_votosArray, partidosArray], 
                              index=['bairro', 'votos', 'partido']).T
    
    dfAgrupadoResultadosPartidoMunicipio['ano'] = [ano]*len(dfAgrupadoResultadosPartidoMunicipio)
    
    return dfAgrupadoResultadosPartidoMunicipio

dfAgrupadoResultadosPartidoMunicipioPorAno = pd.DataFrame()
for a in dfResultadosPartidoMunicipio['ano'].unique():
    dfAgrupadoResultadosPartidoMunicipioPorAno = dfAgrupadoResultadosPartidoMunicipioPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoResultadosPartidoMunicipioPorAno.reset_index(drop=True, inplace=True)

In [ ]:
ano = 2016
fig = px.choropleth_mapbox(dfAgrupadoResultadosPartidoMunicipioPorAno[dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
dfAgrupadoOrientacaoPoliticaMunicipioPorAno = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica', 'latitude', 'longitude']).groupby(level=(0, 1, 2, 3)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
    
dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos totais'] = dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nominais'] + dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nao_nominais']
dfAgrupadoOrientacaoPoliticaMunicipioPorAno['bairro'] = dfAgrupadoOrientacaoPoliticaMunicipioPorAno.apply(lambda l: neighhboorhoodCoordinateIsInside(l[2], l[3]), axis=1)

dfOrientacaoPoliticaMunicipioMaxVotos = pd.DataFrame()
for group, frame in dfAgrupadoOrientacaoPoliticaMunicipioPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_votos = np.max(frame['votos totais'])
    dfOrientacaoPoliticaMunicipioMaxVotos = dfOrientacaoPoliticaMunicipioMaxVotos.append(frame[frame['votos totais'].eq(n_votos)])

dfOrientacaoPoliticaMunicipioMaxVotos.reset_index(inplace=True)

In [ ]:
ano = 2008
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxVotos[dfOrientacaoPoliticaMunicipioMaxVotos['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos totais'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.DataFrame()
    for ano in dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].unique():
        queryFiliacaoPartidaria = """
            SELECT COUNT(*) AS n_filiados, FP.zona, FP.sigla_partido FROM `basedosdados.br_tse_filiacao_partidaria.filiacao_partidaria` AS FP
            WHERE FP.id_municipio_tse={} AND (FP.data_desfiliacao > '{}-12-30' OR (FP.situacao_registro = 'regular' AND FP.data_filiacao < '{}-01-01'))
            GROUP BY FP.zona, FP.sigla_partido
            ORDER BY FP.zona""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0], ano, ano)

        dfResultadoQuery = bd.read_sql(queryFiliacaoPartidaria, billing_project_id='capacitacaoufrjanalytica')
        dfResultadoQuery['ano'] = [ano]*len(dfResultadoQuery)
        dfFiliacaoPartidaria = dfFiliacaoPartidaria.append(dfResultadoQuery)
    dfFiliacaoPartidaria.to_csv("csv/filiados.csv", index=False)
else:
    dfFiliacaoPartidaria = pd.read_csv("csv/filiados.csv")


In [ ]:
filiadosPorPartido = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
partido = ['MDB', 'PT', 'PSDB']
fig = px.line(filiadosPorPartido[filiadosPorPartido['sigla_partido'].isin(partido)], x="ano", y="n_filiados", 
              color = "sigla_partido",
              title='Filiados do '+str(partido)[1:-1]+' X ano')
fig.show()

In [ ]:
dfFiliacaoPartidaria['orientacao politica'] = dfFiliacaoPartidaria['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

In [ ]:
filiadosPorOrientacaoPolitica = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

In [ ]:
fig = px.line(filiadosPorOrientacaoPolitica, x="ano", y="n_filiados", color='orientacao politica',
             title='Filiados de determinada ideologica política X ano')
fig.show()

In [ ]:
dfFiliacaoPartidaria['bairro'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnBairro(x))

In [ ]:
dfFiliacaoPartidaria['latitude'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnLat(x))
dfFiliacaoPartidaria['longitude'] = dfFiliacaoPartidaria['zona'].map(lambda x: returnLon(x))

In [ ]:
def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_filiadosArray = []
    partidosArray = []
    latitudeArray = []
    longitudeArray = []
    for bairro, frame in dfFiliacaoPartidaria[dfFiliacaoPartidaria['ano'].eq(ano)].groupby('bairro'):
        n_filiados = np.max(frame['n_filiados'])
        partido = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['sigla_partido'].values[0]
        latitude = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['latitude'].values[0]
        longitude = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['longitude'].values[0]
            
        n_filiadosArray.append(n_filiados)
        partidosArray.append(partido)
        latitudeArray.append(latitude)
        longitudeArray.append(longitude)
        bairrosArray.append(neighhboorhoodCoordinateIsInside(latitude, longitude))
        
    
    dfAgrupadoFiliacaoPartidaria = pd.DataFrame([bairrosArray, n_filiadosArray, partidosArray], 
                              index=['bairro', 'filiados', 'partido']).T
    
    dfAgrupadoFiliacaoPartidaria['ano'] = [ano]*len(dfAgrupadoFiliacaoPartidaria)
    
    return dfAgrupadoFiliacaoPartidaria

dfAgrupadoFiliacaoPartidariaPorAno = pd.DataFrame()
for a in dfFiliacaoPartidaria['ano'].unique():
    dfAgrupadoFiliacaoPartidariaPorAno = dfAgrupadoFiliacaoPartidariaPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoFiliacaoPartidariaPorAno.reset_index(drop=True, inplace=True)                     

In [ ]:
ano = 1998
fig = px.choropleth_mapbox(dfAgrupadoFiliacaoPartidariaPorAno[dfAgrupadoFiliacaoPartidariaPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['filiados'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
dfAgrupadoFiliacaoOrientacaoPoliticaPorAno = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica', 'latitude', 'longitude']).groupby(level=(0, 1, 2, 3)).agg({"n_filiados":np.sum}).reset_index()
    
dfAgrupadoFiliacaoOrientacaoPoliticaPorAno['bairro'] = dfAgrupadoFiliacaoOrientacaoPoliticaPorAno.apply(lambda l: neighhboorhoodCoordinateIsInside(l[2], l[3]), axis=1)

dfOrientacaoPoliticaMunicipioMaxFiliados = pd.DataFrame()
for group, frame in dfAgrupadoFiliacaoOrientacaoPoliticaPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_filiados = np.max(frame['n_filiados'])
    dfOrientacaoPoliticaMunicipioMaxFiliados = dfOrientacaoPoliticaMunicipioMaxFiliados.append(frame[frame['n_filiados'].eq(n_filiados)])

dfOrientacaoPoliticaMunicipioMaxFiliados.reset_index(inplace=True)

In [ ]:
ano = 2020
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxFiliados[dfOrientacaoPoliticaMunicipioMaxFiliados['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['n_filiados'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

In [ ]:
votosArray = []
anosArray = []
partidosArray = []
cargosArray = []
bairrosArray = []
for group, frame in dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].ne(2)].set_index(['ano', 'sigla_partido', 'cargo', 'bairro']).groupby(level=(0, 1, 2, 3)):
    n_votos = np.sum(frame['votos_nominais']) + np.sum(frame['votos_nao_nominais']) 
    votosArray.append(n_votos)
    anosArray.append(group[0])
    partidosArray.append(group[1])
    cargosArray.append(group[2])
    bairrosArray.append(group[3])

dfVotosAgrupados = pd.DataFrame([votosArray, anosArray, partidosArray, cargosArray, bairrosArray],
                               index=['votos', 'ano', 'partido', 'cargo', 'bairro']).T

In [ ]:
filiadosArray = []
anosArray = []
partidosArray = []
bairrosArray = []
for group, frame in dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido', 'bairro']).groupby(level=(0, 1, 2)):
    n_filiados = np.sum(frame['n_filiados'])
    filiadosArray.append(n_filiados)
    anosArray.append(group[0])
    partidosArray.append(group[1])
    bairrosArray.append(group[2])
    
dfFiliadosAgrupados = pd.DataFrame([filiadosArray, anosArray, partidosArray, bairrosArray],
                               index=['filiados', 'ano', 'partido', 'bairro']).T

In [ ]:
new_df = pd.merge(dfFiliadosAgrupados, dfVotosAgrupados,  how='inner', left_on=['ano', 'partido', 'bairro'], right_on = ['ano', 'partido', 'bairro'])

In [ ]:
new_df['filiados'] = new_df['filiados'].apply(lambda x: int(x)) 
new_df['votos'] = new_df['votos'].apply(lambda x: int(x)) 

In [ ]:
dfWithoutCargos = new_df.set_index(['ano', 'partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"votos":np.sum, "filiados":np.max}).reset_index()

In [ ]:
fig = px.scatter(dfWithoutCargos, x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])
fig.show()

In [ ]:
dfPartidos = pd.DataFrame({'partido': dfWithoutCargos[:-1]['partido'].unique()})
dfPartidos['corr'] = dfPartidos['partido'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['votos'])[0])
dfPartidos.head()

In [ ]:
dfBairros = pd.DataFrame({'bairro': dfWithoutCargos['bairro'].unique()})
dfBairros['corr'] = dfBairros['bairro'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['votos'])[0])

dfBairros.head()

In [ ]:
dfCargos = pd.DataFrame({ 'cargo': new_df['cargo'].unique()})
dfCargos['corr'] = dfCargos['cargo'].map(lambda x: stats.pearsonr(new_df[new_df['cargo'].eq(x)]['filiados'], new_df[new_df['cargo'].eq(x)]['votos'])[0])

dfCargos.head()

In [ ]:
representacao_congresso = pd.read_csv('csv/representacao_congresso.csv')

representacao_congresso.columns = ['partido', 'Assentos no Senado', 'Assentos na Câmara']
representacao_congresso['Assentos no Senado'] = representacao_congresso['Assentos no Senado'].map(lambda x: np.int(x[:-4]))
representacao_congresso['Assentos na Câmara'] = representacao_congresso['Assentos na Câmara'].map(lambda x: np.int(x[:-6]))

representacao_congresso['partido'] = representacao_congresso['partido'].map(lambda x: x.upper())

In [ ]:
dfPartidos = pd.merge(dfPartidos, representacao_congresso,  how='inner', left_on=['partido'], right_on = ['partido'])

In [ ]:
dfPartidos['orientacao politica'] = dfPartidos['partido'].map(lambda x: orientacao_partidos[orientacao_partidos['Nome'].eq(x)]['Congresso em Foco (2019)[37]'].iloc[0])

In [ ]:
def retornarValorDiscretizado(x, minValue, maxValue, bins):
    for i in range(1, bins + 1):
        if x <= minValue + i * ((maxValue - minValue)/bins):
            return str(minValue + (i-1) * ((maxValue - minValue)/bins))+ " - " + str(minValue + i * ((maxValue - minValue)/bins))

def discretizar(i):
    minValue = np.min(i)
    maxValue = np.max(i)
    bins = int(np.sqrt(len(i)))
            
    return i.map(lambda v:retornarValorDiscretizado(v, minValue, maxValue, bins))

dfPartidos['corrDiscretizados'] = discretizar(dfPartidos['corr'])

In [ ]:
def calcularIndiceDeGiniParaValor(df, resultado, valor, coluna):
    gini = 0
    for value in df[resultado].unique():
        pi = len(df[df[coluna].eq(valor) & df[resultado].eq(value)])/len(df[df[coluna].eq(valor)])
        gini += pi * pi
    gini = 1 - gini
    return gini
    
def calcularIndiceDeGini(df, resultado, coluna):
    values = df[coluna].unique()
    gini = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        gini += calcularIndiceDeGiniParaValor(df, resultado, i, coluna) * pi
    
    return gini

In [ ]:
partidosGini = pd.DataFrame()

for c in ['orientacao politica', 'Assentos no Senado', 'Assentos na Câmara']:
    partidosGini = partidosGini.append({'coluna': c, 
                                        'gini': calcularIndiceDeGini(dfPartidos, 'corrDiscretizados', c)}, 
                                       ignore_index=True)

In [ ]:
fig = px.bar(partidosGini, x='coluna', y='gini', color='coluna',
            title='Indice de gini por coluna dos Partidos')
fig.show()

In [ ]:
partidosCorrMediaOrientacaoPolitica = dfPartidos.groupby('orientacao politica').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaOrientacaoPolitica, x='orientacao politica', y='corr', color='orientacao politica',
            title='Correlação media por orientacao politica')
fig.show()

In [ ]:
partidosCorrMediaAssentosNoSenado = dfPartidos.groupby('Assentos no Senado').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaAssentosNoSenado, x='Assentos no Senado', y='corr', color='Assentos no Senado',
            title='Correlação media por Assentos no Senado')
fig.show()

In [ ]:
partidosCorrMediaAssentosNoSenadoCongresso = dfPartidos.groupby('Assentos na Câmara').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaAssentosNoSenadoCongresso, x='Assentos na Câmara', y='corr', color='Assentos na Câmara',
            title='Correlação media por Assentos na Câmara')
fig.show()